In [6]:
%pip install "opencompass[full]"
%pip install pytorch transformers datasets "opencompass[full]"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://mirrors.aliyun.com/pypi/simple
  Using cached http://mirrors.aliyun.com/pypi/packages/e4/55/36702adfc9af939e7aee972c6f1bb8e514863af9e554af180a5459efc67e/opencompass-0.3.7-py3-none-any.whl (2.4 MB)
  Using cached http://mirrors.aliyun.com/pypi/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached http://mirrors.aliyun.com/pypi/packages/af/84/1831ce6ffa87b8fd4d9673c3595d0fc4e6631c0691eb43f406d3bf89b951/cpm_kernels-1.0.11-py3-none-any.whl (416 kB)
  Using cached http://mirrors.aliyun.com/pypi/packages/18/d7/ed1ce1d5e00b0cd0e1ca46a710eb00822add013048c733d5b82db490e643/einops-0.5.0-py3-none-any.whl (36 kB)
  Using cached http://mirrors.aliyun.com/pypi/packages/a2/e7/cbca9e2d2590eb9b5aa8f7ebabe1beb1498f9462d2ecede5c9fd9735faaf/evaluate-0.4.3-py3-none-any.whl (84 kB)
  Using cached func_timeout-4.3.5-py3-none-any.whl
  Using cached http://mirrors.aliyun.com/pypi/packages/43/ff/74f23998ad2f9

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://mirrors.aliyun.com/pypi/simple
  Using cached http://mirrors.aliyun.com/pypi/packages/ee/67/f403d4ae6e9cd74b546ee88cccdb29b8415a9c1b3d80aebeb20c9ea91d96/pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-8qms25t5/pytorch_91dbd31417394ec0affab3ac1b392011/setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERR

安装opencompass：Kaggle上已经为我们准备好了其他常用包，只需安装opencompass用于评测即可。如果不在Kaggle上运行，则还需要安装其他必要包。

# 指令微调

In [1]:
"""
The main program for finetuning LLMs with Huggingface Transformers Library.

ALL SECTIONS WHERE CODE POSSIBLY NEEDS TO BE FILLED IN ARE MARKED AS TODO.
"""

import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets

In [2]:
# Define the arguments required for the main program.
# NOTE: You can customize any arguments you need to pass in.
@dataclass
class ModelArguments:
    """Arguments for model
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."
        }
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype."
            ),
            "choices": ["bfloat16", "float16", "float32"],
        },
    )
    # TODO: add your model arguments here
    pass


@dataclass
class DataArguments:
    """Arguments for data
    """
    dataset_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."
        }
    )
    # TODO: add your data arguments here

In [17]:
##### Attempt & Debug #####
dataset = datasets.load_dataset('./Dataset/alpaca-language-instruction-training')
instructions = dataset['train']['instruction']
inputs = dataset['train']['input']
outputs = dataset['train']['output']
dataset['train'][0]

{'instruction': 'Give three tips for staying healthy.',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'input': None}

In [17]:
##### Dataset tokenization test
dataset = datasets.load_dataset('./Dataset/alpaca-language-instruction-training')
tokenizer = AutoTokenizer.from_pretrained('./model/input/qwen2.5/transformers/0.5b/1')
model = AutoModelForCausalLM.from_pretrained('./model/input/qwen2.5/transformers/0.5b/1',device_map = 'auto')
tokenized_input = tokenizer(
            [f"Instructions:{item['instruction']},input:{'' if item['input']==None else item['input']},Outputs:{item['output']}" 
             for item in dataset['train']],
            padding=True,
            padding_side='left',
            truncation=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
print(tokenized_input)
print(tokenized_input['input_ids'].size())
# print(tokenized_input['labels'].size())

{'input_ids': tensor([[151643, 151643, 151643,  ...,   1817,   3729,     13],
        [151643, 151643, 151643,  ...,    320,  18184,    568],
        [151643, 151643, 151643,  ...,  19124,    374,     13],
        ...,
        [151643, 151643, 151643,  ...,  61438,     25,   2514],
        [151643, 151643, 151643,  ...,     25,   3707,   4014],
        [151643, 151643, 151643,  ...,    264,   2097,     13]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])}
torch.Size([51760, 1031])


In [19]:
input_datasets = tokenized_datasets.remove_columns(['instruction', 'input', 'output'])
input_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 51760
})

In [20]:
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained('./model/input/qwen2.5/transformers/0.5b/1')
model = AutoModelForCausalLM.from_pretrained('./model/input/qwen2.5/transformers/0.5b/1',device_map = 'auto')
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [21]:
### test for inference.

input_seqs = ["Instructions: Tell me the climate in Beijing, the captial of China.",
              "Instructions: Describe president Blinken of the US."]
inputs = tokenizer(input_seqs, padding=True,padding_side='left',return_tensors="pt").to(device)
generated_ids = model.generate(
    **inputs,
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=2048,
    top_p=0.95,
    temperature=0.9,
    do_sample=True
)

generated_text = tokenizer.batch_decode(
    generated_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)
print(generated_text)

['Instructions: Tell me the climate in Beijing, the captial of China. Climate Overview: Beijing is a densely populated and industrialized city located on the northwestern edge of the Chinese mainland. It is the capital of China and is considered the largest city in the country by population. The climate in Beijing can vary greatly depending on location and season, but overall, Beijing has a subtropical climate, characterized by hot and humid summers and cold and dry winters. The average temperature in Beijing is around 25-32°C (77-89°F) during the summer months, with highs reaching up to 40-45°C (104-113°F) during the summer. Winter in Beijing is characterized by cold and snowy winters, with average temperatures typically ranging from 0°C (32°F) in the winter to -5°C (23°F) during the winter. The average temperature in Beijing in the winter is around -5°C (23°F), with low temperatures of -10°C (14°F) during the winter and high temperatures of 0°C (32°F) during the winter. Overall, Beij

In [3]:
# The main function
# NOTE You can customize some logs to monitor your program.
def finetune():
    # TODO Step 1: Define an arguments parser and parse the arguments
    # NOTE Three parts: model arguments, data arguments, and training arguments
    # HINT: Refer to 
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/internal/trainer_utils#transformers.HfArgumentParser
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/trainer#transformers.TrainingArguments
    parser = HfArgumentParser((ModelArguments, DataArguments,TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses(args=sys.argv)

    # TODO Step 2: Load tokenizer and model
    # HINT 1: Refer to
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/main_classes/tokenizer#tokenizer
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/qwen2
    # HINT 2: To save training GPU memory, you need to set the model's parameter precision to half-precision (float16 or bfloat16).
    #         You may also check other strategies to save the memory!
    #   * https://huggingface.co/docs/transformers/v4.46.3/en/model_doc/llama2#usage-tips
    #   * https://huggingface.co/docs/transformers/perf_train_gpu_one
    #   * https://www.53ai.com/news/qianyanjishu/2024052494875.html
    device = 'cuda'
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
    model = AutoModelForCausalLM.from_pretrained(model_args.model_name_or_path,device_map = 'auto',torch_dtype = 'bfloat16')

    # TODO Step 3: Load dataset
    # HINT: https://huggingface.co/docs/datasets/v3.1.0/en/package_reference/main_classes#datasets.Dataset
    dataset = datasets.load_dataset(data_args.dataset_path)
    raw_train_dataset = dataset['train']
    
    # TODO Step 4: Define the data collator function
    # NOTE During training, for each model parameter update, we fetch a batch of data, perform a forward and backward pass,
    # and then update the model parameters. The role of the data collator is to process the data (e.g., padding the data within
    # a batch to the same length) and format the batch into the input required by the model.
    #
    # In this assignment, the purpose of the custom data_collator is to process each batch of data from the dataset loaded in
    # Step 3 into the format required by the model. This includes tasks such as tokenizing the data, converting each token into 
    # an ID sequence, applying padding, and preparing labels.
    # 
    # HINT:
    #   * Before implementation, you should:
    #      1. Clearly understand the format of each sample in the dataset loaded in Step 3.
    #      2. Understand the input format required by the model (https://huggingface.co/docs/transformers/model_doc/qwen2#transformers.Qwen2ForCausalLM).
    #         Reading its source code also helps!

    def data_collator(batch: List[Dict]):
        """
        batch: list of dict, each dict of the list is a sample in the dataset.
        """
        tokenized_input = tokenizer(
            [f"Instructions:{item['instruction']},input:{'' if item['input']==None else item['input']}" 
             for item in batch],
            padding='max_length',
            max_length=1024,
            padding_side='left',
            truncation=True,
            add_special_tokens=True,
            return_tensors='pt'
        ).to(device)
        tokenized_output = tokenizer(
            [f"Instructions:{item['instruction']},input:{'' if item['input']==None else item['input']}.Output: {item['output']}" for item in batch],
            padding='max_length',
            max_length=1024,
            padding_side='left',
            truncation=True,
            add_special_tokens=True,
            return_tensors='pt'
        ).to(device)
        
        input_ids,attention_mask = tokenized_input['input_ids'],tokenized_input['attention_mask']
        labels = tokenized_output['input_ids']
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

    # TODO Step 5: Define the Trainer
    # HINT: https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        data_collator=data_collator,
        train_dataset=raw_train_dataset
    )

    # Step 6: Train!
    trainer.train()

In [4]:
# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
base_dir = './model/input/qwen2.5/transformers/0.5b/1'
sys.argv = [
    "--model_name_or_path", base_dir,
    "--torch_dtype", "bfloat16",
    "--output_dir","./output",
    "--overwrite_output_dir","True",
    "--dataset_path","./Dataset/alpaca-language-instruction-training",
    "--remove_unused_columns", "False",
    "--per_device_train_batch_size", "4",
    "--do_train", "True",
    "--dataloader_pin_memory","False",
]
finetune()

Generating train split: 0 examples [00:00, ? examples/s]

/tmp/ipykernel_2075/1767682112.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,1.261200
1000,1.145800
1500,1.132100
2000,1.097800
2500,1.140200
3000,1.080800
3500,1.095000
4000,1.037900
4500,1.088700
5000,1.037500


# 评测模型

如果你有多个GPU，可以修改下面的--hf-num-gpus参数来加速评测.

In [12]:
!opencompass \
    --datasets mmlu_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path  output/checkpoint-38820 \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 600 \
    --batch-size 4 \
    --hf-num-gpus 1 \
    --work-dir "./eval" \
    --debug

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/__init__.py:19: UserWarning: Starting from v0.4.0, all AMOTIC configuration files currently located in `./configs/datasets`, `./configs/models`, and `./configs/summarizers` will be migrated to the `opencompass/configs/` package. Please update your configuration file paths accordingly.
  _warn_about_config_migration()
12/09 21:55:26 - OpenCompass - INFO - Loading mmlu_ppl: /root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/configs/./datasets/mmlu/mmlu_ppl.py
12/09 21:55:27 - OpenCompass - INFO - Loading example: /root/miniconda3/envs/nlp/lib/python3.10/site-packages/opencompass/configs/./summarizers/example.py
12/09 21:55:27 - OpenCompass - INFO - Current exp folder: ./eval/20241209_215527
12/09 21:55:27 - OpenCompass - WARNING - SlurmRunner is not used, so the partition argument is ignored.
12/09 21:55:27 - OpenCompass - INFO - Partitioned into 1 tasks.
12/09 21:55:31 - OpenCompass - INFO - Task [checkpo

In [ ]:
%opencompass \
    --datasets mmlu_ppl hellaswag_clean_ppl winogrande_ll ARC_e_ppl ARC_c_clean_ppl SuperGLUE_BoolQ_few_shot_ppl \
    --summarizer example \
    --hf-type base \
    --hf-path {SFT_MODEL_PATH} \
    --tokenizer-kwargs padding_side="left" truncation="left" \
    --max-seq-len 2048 \
    --batch-size 4 \
    --hf-num-gpus 1 \
    --work-dir "/kaggle/working/evals/sft" \
    --debug